In [ ]:
class Hand:
    def __init__(self):
        self.cards = []
        self.evaluate()
        
    
    def evaluate(self):
        self.value = sum([card.value for card in self.cards])
        self.aces = [card.rank for card in self.cards].count('A')
        while self.value > 21 and self.aces > 0:
            self.value -= 10
            self.aces -= 1
    
    def add_card(self, card):
        self.cards.append(card)
        self.evaluate()
        return self.value
    
    def is_blackjack(self):
        return self.value == 21 and len(self.cards) == 2

    def __repr__(self):
        return f'<Hand: {self.value} {[str(card) for card in self.cards]}>'
    
    

In [ ]:
# HOUSE RULES
SPLITTING_SAME_VALUE = True # if you can split same-valued but different-ranked cards


In [ ]:
class Game:
    def __init__(self, d, number_decks=6):
        self.deck = Deck(number_decks)
        self.deck.shuffle()
        self.d = d
        self.d.hit_button.on_click(self.hit)
        self.d.stand_button.on_click(self.stand)
        print(f'New game with {self.deck}')

        self.payouts = [0]

    def dealer_draw_card(self):
        card = self.deck.deal()
        print(f"Dealer draws {card}")
        self.dealer.add_card(card)
        self.d.update_deck_info(self.deck)
        self.d.dealer_cards_label.value = str([str(card) for card in self.dealer.cards])
        self.d.dealer_value_label.value = str(self.dealer.value)
    
    def player_draw_card(self):
        card = self.deck.deal()
        print(f"Player draws {card}")
        self.player.add_card(card)
        self.d.update_deck_info(self.deck)
        self.d.player_cards_label.value = str([str(card) for card in self.player.cards])
        self.d.player_value_label.value = str(self.player.value)

    def player_turn(self):
        self.d.activation(0,1)
        self.player_draw_card()
        self.player_draw_card()

        if self.player.is_blackjack():
            self.d.player_cards_label.style.text_color = 'gold'
            self.d.player_cards_label.style.font_weight = 'bolder'
            print("Player has a Blackjack!")
            self.finish_round()

        if (self.player.cards[0].rank == self.player.cards[1].rank) or (SPLITTING_SAME_VALUE and self.player.cards[0].value == self.player.cards[1].value):
            print("Player can split!")
            self.d.split_button.disabled = False
            # TODO: implement splitting
        
        
        
        
        if self.player.value < 21:
            self.d.double_button.disabled = False
            # TODO: implement doubling down
            self.d.hit_button.disabled = False
            self.d.stand_button.disabled = False

    def hit(self, b):
        with self.d.output:
            self.d.split_button.disabled = True
            self.d.double_button.disabled = True
            self.player_draw_card()
            if self.player.value > 21:
                print("Player busts!")
                self.d.player_value_label.style.text_color = 'red'
                self.d.player_value_label.style.font_weight = 'bolder'
                self.d.hit_button.disabled = True
                self.d.stand_button.disabled = True
                self.finish_round()

            elif self.player.value == 21:
                self.d.player_value_label.style.font_weight = 'bolder'
                print("Player stands at 21!")
                self.d.hit_button.disabled = True
                self.d.stand_button.disabled = True
                self.finish_round()

    def stand(self, b):
        with self.d.output:
            self.d.disable_action_buttons()
            self.d.player_value_label.style.font_weight = 'bolder'
            print("Player stands at", self.player.value)
            self.finish_round()
           
    def finish_round(self):
        self.d.activation(1,0)
        self.dealer_turn()
        self.d.activation(0,0)

        payout = self.calculate_payout(self.player)
        print(f'Payout: {payout}')
        self.payouts.append(payout)

        self.d.update_payout_graph(self.payouts)

        self.d.disable_action_buttons()
        self.d.deal_button.disabled = False
        

    
    def dealer_turn(self):
        self.d.activation(1,0)
        while self.dealer.value < 17:
            self.dealer_draw_card()
            if self.dealer.value > 21:
                self.d.dealer_value_label.style.text_color = 'red'
                self.d.dealer_value_label.style.font_weight = 'bolder'
                break
            elif self.dealer.value >= 17:
                if self.dealer.is_blackjack():
                    self.d.dealer_cards_label.style.text_color = 'gold'
                    self.d.dealer_cards_label.style.font_weight = 'bolder'
                    print("Dealer has a Blackjack!")
                else:
                    self.d.dealer_value_label.style.font_weight = 'bolder'
                    print("Dealer stands at", self.dealer.value)
                break

    def calculate_payout(self, player_hand):
        if player_hand.is_blackjack():
            if self.dealer.is_blackjack():
                print("Player and Dealer have a Blackjack!")
                print("Push!")
                return 0
            else: 
                self.d.dealer_value_label.style.text_color = 'red'
                self.d.player_value_label.style.text_color = 'green'
                print("Player wins with a Blackjack!")
                return 1.5
        
        if player_hand.value > 21:
            print("Player busts!")
            self.d.dealer_value_label.style.text_color = 'green'
            return -1
        if self.dealer.value > 21:
            self.d.player_value_label.style.text_color = 'green'
            print("Dealer busts!")
            return 1

        
        
        if self.dealer.value == player_hand.value:
            print("Push!")
            return 0
        
        if self.dealer.value > player_hand.value:
            self.d.dealer_value_label.style.text_color = 'green'
            self.d.player_value_label.style.text_color = 'red'
            print("Dealer wins!")
            return -1
        
        if player_hand.value > self.dealer.value:
            self.d.dealer_value_label.style.text_color = 'red'
            self.d.player_value_label.style.text_color = 'green'
            print("Player wins!")
            return 1

    def play_round(self):
        # setup
        self.d.reset_colors()
        self.player = Hand()
        self.dealer = Hand()

        # dealer first card
        self.d.activation(1,0)
        self.dealer_draw_card()
        
        # player
        self.d.activation(0,1)
        self.player_turn()
